In [21]:
#Lets see how to predict stock price using ML techniques
# import necessary libraries
import numpy as np 
import pandas as pd  
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
# import the data using pandas 
df = pd.read_csv('../Data/dataset.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-02-14,392.932007,407.230011,392.932007,407.230011,407.230011,74469800.0
1,2016-02-15,407.567993,410.381012,397.748993,400.184998,400.184998,74070496.0
2,2016-02-16,401.432007,408.945007,401.432007,407.488007,407.488007,73093104.0
3,2016-02-17,407.656006,421.166992,406.783997,416.321991,416.321991,83193600.0
4,2016-02-18,416.571991,425.996002,415.638000,422.372986,422.372986,76752600.0


In [23]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1824.000000,1824.000000,1824.000000,1824.000000,1824.000000,1.824000e+03
mean,6997.507543,7193.644833,6799.439468,7021.818023,7021.818023,1.314212e+10
std,6643.642365,6907.088807,6387.026749,6704.093976,6704.093976,1.632261e+10
min,392.932007,407.230011,392.932007,400.184998,400.184998,2.851400e+07
25%,1795.737488,1850.485015,1718.399994,1807.910034,1807.910034,7.868065e+08
50%,6572.784912,6674.324952,6453.779785,6578.659912,6578.659912,5.864455e+09
75%,9361.835449,9540.433594,9231.055664,9375.034424,9375.034424,2.026759e+10
max,48483.144531,49485.171875,47127.769531,48832.718750,48832.718750,1.233206e+11


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1828 entries, 0 to 1827
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1828 non-null   object 
 1   Open       1824 non-null   float64
 2   High       1824 non-null   float64
 3   Low        1824 non-null   float64
 4   Close      1824 non-null   float64
 5   Adj Close  1824 non-null   float64
 6   Volume     1824 non-null   float64
dtypes: float64(6), object(1)
memory usage: 100.1+ KB


In [25]:
df = df.dropna(inplace=True)